# 4 - External Validation with Kenema Data

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from scipy import interp
from scipy.interpolate import interp1d

# Use the RWinOut instead of rpy2.ipython to get output on windows 
# https://bitbucket.org/rpy2/rpy2/issues/125/set_writeconsole-not-working-on-windows
# https://github.com/vitorcurtis/RWinOut
#%load_ext RWinOut
%load_ext rpy2.ipython

%matplotlib inline

In [2]:
# %load import_notebook.py
# Infraestructure to import a Jupyter notebook
# http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [3]:
from LogRegUtils import LogRegModel
from LogRegUtils import caldis, calibration_table, calibration, calibration2
from LogRegUtils import create_plots

importing Jupyter notebook from LogRegUtils.ipynb


In [4]:
sel_model = 2

risk_threshold = 0.5

make_plots = False

if sel_model == 1:
    model_name = 'all-minimal'
elif sel_model == 2:
    model_name = 'all-clinical-only'
elif sel_model == 3:
    model_name = 'all-parsimonious'    
elif sel_model == 4:
    model_name = 'all-parsimonious-notemp'

In [5]:
kenema_data_file = '../../kenema/mirador/data_normalized.csv'
kenema_data = pd.read_csv(kenema_data_file, na_values="\\N")

var_dict = { 
  'Disposition':'OUT',
  'PatientAge':'AGE', 
  'cycletime':'CT', 
  'FeverTemperature':'TEMP', 
  'Fever':'FEVER',
  'Jaundice':'JAUN', 
  'Bleeding':'BLEED', 
  'Breathlessness':'BREATH', 
  'SwallowingProblems':'STHROAT', 
  'AstheniaWeakness':'WEAK',
  'reftime':'DONSET',
  'Diarrhoea':'DIARR'
}

In [6]:
model_params = os.path.join(model_name, 'model.csv')
model = LogRegModel(model_params)

predictors = model.getPredictors()

variables = ['OUT'] + [var_dict[var] for var in predictors]

test_data = kenema_data[variables]
complete_data = test_data.dropna()
print(complete_data)

     OUT   AGE  TEMP  JAUN  BLEED  BREATH  STHROAT  WEAK  DIARR
20   1.0  38.0  36.7     0    0.0     0.0      0.0   0.0    1.0
21   1.0  43.0  37.1     0    0.0     0.0      0.0   1.0    1.0
23   1.0  37.0  38.4     0    0.0     0.0      0.0   1.0    0.0
26   1.0  45.0  36.4     0    0.0     0.0      1.0   1.0    1.0
28   1.0  35.0  38.3     0    0.0     0.0      1.0   1.0    1.0
29   1.0  22.0  36.2     0    0.0     0.0      0.0   1.0    1.0
30   1.0  50.0  38.4     0    0.0     1.0      1.0   1.0    0.0
31   1.0  14.0  37.0     0    0.0     0.0      0.0   0.0    0.0
32   1.0  45.0  39.9     0    0.0     1.0      0.0   1.0    0.0
37   1.0  60.0  37.4     0    0.0     0.0      1.0   1.0    0.0
38   1.0  35.0  36.1     0    0.0     0.0      0.0   0.0    1.0
39   0.0  50.0  36.2     0    0.0     0.0      0.0   0.0    1.0
42   0.0  45.0  36.4     0    0.0     0.0      0.0   0.0    0.0
46   1.0  63.0  37.4     0    0.0     0.0      0.0   0.0    0.0
48   0.0  36.0  36.3     0    0.0     0.

### Results on complete data

In [7]:
x = complete_data[complete_data.columns[1:]].values
ytrue = [int(v) for v in complete_data[complete_data.columns[0]].values]
probs = model.predict(x)
ypred = [int(risk_threshold < p) for p in probs]

auc = roc_auc_score(ytrue, probs)
fpr, tpr, thresholds = roc_curve(ytrue, probs) 
brier = brier_score_loss(ytrue, probs)
cal, dis = caldis(ytrue, probs)
acc = accuracy_score(ytrue, ypred)
precision, recall, f1score, support = precision_recall_fscore_support(ytrue, ypred)

P = N = 0
TP = TN = 0
FP = FN = 0
for i in range(len(ytrue)):
    if ytrue[i] == 1:
        P += 1
        if ypred[i] == 1: TP += 1
        else: FN += 1
    else:
        N += 1
        if ypred[i] == 0: TN += 1
        else: FP += 1
            
sens = float(TP)/P
spec = float(TN)/N

# Positive and Negative Predictive Values
# https://en.wikipedia.org/wiki/Positive_and_negative_predictive_values
ppv = float(TP) / (TP + FP)
npv = float(TN) / (TN + FN)
        
# Likelihood ratios
# https://en.wikipedia.org/wiki/Likelihood_ratios_in_diagnostic_testing
lr_pos = sens / (1 - spec) if spec < 1 else np.inf
lr_neg = (1 - sens) / spec if 0 < spec else np.inf
    
# print "True outcomes:", ytrue
# print "Prediction   :", ypred
cfr = 100 * (float(np.sum(ytrue)) / len(ytrue))
print("Number of cases :", len(ytrue))
print("Number of deaths:", np.sum(ytrue)) 
print("CFR             : %0.2f" % cfr)

print("")
print("Measures of performance") 
print("AUC           : %0.2f" % auc) 
print("Brier         : %0.2f" % brier) 
# print("Calibration   :", cal) 
# print("Discrimination:", dis) 
print("Accuracy      : %0.2f" % acc) 
print("Sensitivity   : %0.2f" % sens) 
print("Specificity   : %0.2f" % spec) 
# print("PPV           :", ppv) 
# print("NPV           :", npv) 
# print("LR+           :", lr_pos) 
# print("LR-           :", lr_neg) 

# print("Precision (live)  :", precision[0]," (specificity for die)")
# print("Precision (die)   :", precision[1]," (specificity for live)")
# print("Sensitivity (live):", recall[0])
# print("Sensitivity (die) :", recall[1]) 
# print("F1 (live)         :", f1score[0]) 
# print("F1 (die)          :", f1score[1])

with open(os.path.join(model_name, 'kenema-validation.txt'), 'w') as of:
    of.write("Measures of performance\n")
    of.write("AUC           : %0.2f\n" % auc)
    of.write("Brier         : %0.2f\n" % brier)
#     of.write("Calibration   : " + str(cal) + "\n")
#     of.write("Discrimination: " + str(dis) + "\n")
    of.write("Accuracy      : %0.2f\n" % acc)
    of.write("Sensitivity   : %0.2f\n" % sens)
    of.write("Specificity   : %0.2f\n" % spec)
#     of.write("PPV           : " + str(ppv) + "\n")
#     of.write("NPV           : " + str(npv) + "\n")
#     of.write("LR+           : " + str(lr_pos) + "\n")
#     of.write("LR-           : " + str(lr_neg) + "\n")    
    
#     of.write("Precision (live)  : " + str(precision[0]) + " (specificity for die)\n")
#     of.write("Precision (die)   : " + str(precision[1]) + " (specificity for live)\n")
#     of.write("Sensitivity (live): " + str(recall[0]) + "\n")
#     of.write("Sensitivity (die) : " + str(recall[1]) + "\n")
#     of.write("F1 (live)         : " + str(f1score[0]) + "\n")
#     of.write("F1 (die)          : " + str(f1score[1]) + "\n")

Number of cases : 44
Number of deaths: 36
CFR             : 81.82

Measures of performance
AUC           : 0.72
Brier         : 0.22
Accuracy      : 0.59
Sensitivity   : 0.56
Specificity   : 0.75


In [41]:
if make_plots:
    fig, ax = plt.subplots()
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.plot([0, 1], [0, 1], 'k--', c='grey')
    plt.plot(fpr, tpr, color='black')
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')
    fig.savefig(os.path.join(model_name, 'kenema-roc-complete.pdf'))

    cal_table = calibration_table(ytrue, probs, 10)
    fig, ax = plt.subplots()
    plt.plot([0.05, 0.95], [0.05, 0.95], '-', c='grey', linewidth=0.5, zorder=1)
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.xlabel('Predicted Risk')
    plt.ylabel('Observed Risk')
    x = cal_table['pred_prob']
    y = cal_table['true_prob']
    # f = interp1d(x, y, kind='cubic')
    # xnew = np.linspace(min(x), max(x), num=50, endpoint=True)    
    # plt.plot(xnew, f(xnew))
    plt.plot(x, y, color='black')
    fig.savefig(os.path.join(model_name, 'kenema-cal-complete.pdf'))

### Results on imputed data

In [15]:
# Source data
test_data_folder = 'imp-kenema'
test_data_file = os.path.join(model_name, 'src_kenema.csv')

num_imp = 100   # Number of multiple imputations

# Load imputation files for selected model, if any
from os import listdir, makedirs
from os.path import isfile, join, exists    
imp_data_folder = os.path.join(model_name, test_data_folder)
if not os.path.exists(imp_data_folder):
    os.makedirs(imp_data_folder)
imp_data_files = [join(imp_data_folder, f) for f in listdir(imp_data_folder) if isfile(join(imp_data_folder, f))]

test_data.to_csv(test_data_file, index=False, na_rep="\\N")

In [ ]:
%%R -i num_imp,imp_data_folder,test_data_file -o imp_data_files

# Imputation in R using MICE
library(mice)

random_seed <- 151
set.seed(random_seed)

src_data <- read.table(test_data_file, sep=",", header=TRUE, na.strings="\\N")

imp_data <- mice(src_data, meth='pmm', m=num_imp)
var_drop <- c(".imp", ".id")
imp_data_files <- character(0)
for (iter in 1:num_imp) {
    comp_data <- complete(imp_data, action=iter)  
    comp_data <- comp_data[,!(names(comp_data) %in% var_drop)]
    fn <- paste(imp_data_folder, "/imputation-", iter, ".csv", sep="")
    write.csv(comp_data, file=fn, row.names=FALSE)
    imp_data_files <- c(imp_data_files, fn)
}

In [11]:
ytrue_all = []
probs_all = []

cfr_list = []
auc_list = [] 
brier_list = [] 
cal_list = [] 
dis_list = [] 
acc_list = []
prec0_list = [] 
prec1_list = [] 
rec0_list = [] 
rec1_list = []
f10_list = [] 
f11_list = []  

for fn in imp_data_files:
    data = pd.read_csv(fn)

    x = data[data.columns[1:]].values
    ytrue = [int(v) for v in data[data.columns[0]].values]
    probs = list(model.predict(x))
    
    ypred = [int(0.5 < p) for p in probs]
    cfr = float(np.sum(ytrue)) / len(ytrue)
    
    ytrue_all += ytrue
    probs_all += probs
    
    auc = roc_auc_score(ytrue, probs)
    brier = brier_score_loss(ytrue, probs)
    cal, dis = caldis(ytrue, probs)
    acc = accuracy_score(ytrue, ypred)
    precision, recall, f1score, support = precision_recall_fscore_support(ytrue, ypred)
    
    cfr_list.append(cfr)
    auc_list.append(auc)
    brier_list.append(brier)
    cal_list.append(cal)
    dis_list.append(dis)
    acc_list.append(acc)
    prec0_list.append(precision[0])
    prec1_list.append(precision[1])
    rec0_list.append(recall[0])
    rec1_list.append(recall[1])
    f10_list.append(f1score[0])
    f11_list.append(f1score[1])  

cfr_mean = np.mean(cfr_list)
auc_mean = np.mean(auc_list)
brier_mean = np.mean(brier_list)
cal_mean = np.mean(cal_list)
dis_mean = np.mean(dis_list)      
acc_mean = np.mean(acc_list)
prec0_mean = np.mean(prec0_list)
prec1_mean = np.mean(prec1_list)
rec0_mean = np.mean(rec0_list)
rec1_mean = np.mean(rec1_list)
f10_mean = np.mean(f10_list)
f11_mean = np.mean(f11_list)
 
cfr_dev = np.std(cfr_list)
auc_dev = np.std(auc_list)
brier_dev = np.std(brier_list)
cal_dev = np.std(cal_list)
dis_dev = np.std(dis_list)      
acc_dev = np.std(acc_list)
prec0_dev = np.std(prec0_list)
prec1_dev = np.std(prec1_list)
rec0_dev = np.std(rec0_list)
rec1_dev = np.std(rec1_list)
f10_dev = np.std(f10_list)
f11_dev = np.std(f11_list)    
    
print("Number of cases :", len(ytrue))
print("Mean CFR        : %0.2f" % (100 * cfr_mean))
print("")
print("Measures of performance")
print("AUC           : %0.2f (%0.2f)" % (auc_mean, auc_dev))
print("Brier         : %0.2f (%0.2f)" % (brier_mean, brier_dev))
# print("Calibration   :", cal_mean, '+/-', cal_dev)
# print("Discrimination:", dis_mean, '+/-', dis_dev)
print("Accuracy      : %0.2f (%0.2f)" % (acc_mean, acc_dev))
print("Sensitivity   : %0.2f (%0.2f)" % (rec1_mean, rec1_dev))
print("Specificity   : %0.2f (%0.2f)" % (prec1_mean, prec1_dev))

# print("Precision (live)  :", prec0_mean, '+/-', prec0_dev," (specificity for die)")
# print("Precision (die)   :", prec1_mean, '+/-', prec1_dev," (specificity for live)") 
# print("Sensitivity (live):", rec0_mean, '+/-', rec0_dev) 
# print("Sensitivity (die) :", rec1_mean, '+/-', rec1_dev) 

with open(os.path.join(model_name, 'kenema-validation-imp.txt'), 'w') as of:
    of.write("Measures of performance\n")
    of.write("AUC           : %0.2f (%0.2f)\n" % (auc_mean, auc_dev))
    of.write("Brier         : %0.2f (%0.2f)\n" % (brier_mean, brier_dev))
#     of.write("Calibration   : " + str(cal_mean) + "+/-" + str(cal_dev) + "\n")
#     of.write("Discrimination: " + str(dis_mean) + "+/-" + str(dis_dev) + "\n")
    of.write("Accuracy      : %0.2f (%0.2f)\n" % (acc_mean, acc_dev))
    of.write("Sensitivity   : %0.2f (%0.2f)\n" % (rec1_mean, rec1_dev))    
    of.write("Specificity   : %0.2f (%0.2f)\n" % (prec1_mean, prec1_dev))    
    
#     of.write("Precision (live)  : " + str(prec0_mean) + "+/-" + str(prec0_dev) + " (specificity for die)\n")
#     of.write("Precision (die)   : " + str(prec1_mean) + "+/-" + str(prec1_dev) + " (specificity for live)\n")
#     of.write("Sensitivity (live): " + str(rec0_mean) + "+/-" + str(rec0_dev) + "\n")
#     of.write("Sensitivity (die) : " + str(rec1_mean) + "+/-" + str(rec1_dev) + "\n")

Number of cases : 106
Mean CFR        : 74.00

Measures of performance
AUC           : 0.78 (0.04)
Brier         : 0.19 (0.01)
Accuracy      : 0.73 (0.03)
Sensitivity   : 0.73 (0.03)
Specificity   : 0.88 (0.03)
